In [19]:
# Install torch-geometric and its dependencies
!pip install torch-scatter torch-sparse torch-geometric -f https://data.pyg.org/whl/torch-2.0.0+cpu.html

Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html


In [20]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [21]:
import torch 
device = torch.device("cpu")  # Set device to CPU

Cleaning the dataset and data preprocessing

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
import polars as pl
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, log_loss
import gc


# Load the dataset using polars for better efficiency
train_schema = {
    'id': pl.UInt64,
    'click': pl.Int8,
    'hour': pl.String,
    'C1': pl.Int16,
    'banner_pos': pl.Int8,
    'site_id': pl.Categorical,
    'site_domain': pl.Categorical,
    'site_category': pl.Categorical,
    'app_id': pl.Categorical,
    'app_domain': pl.Categorical,
    'app_category': pl.Categorical,
    'device_id': pl.Categorical,
    'device_ip': pl.Categorical,
    'device_model': pl.Categorical,
    'device_type': pl.Int8,
    'device_conn_type': pl.Int8,
    'C14': pl.Int16,
    'C15': pl.Int16,
    'C16': pl.Int16,
    'C17': pl.Int16,
    'C18': pl.Int16,
    'C19': pl.Int16,
    'C20': pl.Int32,
    'C21': pl.Int16
}

test_schema = {
    'id': pl.UInt64,
    'hour': pl.String,
    'C1': pl.Int16,
    'banner_pos': pl.Int8,
    'site_id': pl.Categorical,
    'site_domain': pl.Categorical,
    'site_category': pl.Categorical,
    'app_id': pl.Categorical,
    'app_domain': pl.Categorical,
    'app_category': pl.Categorical,
    'device_id': pl.Categorical,
    'device_ip': pl.Categorical,
    'device_model': pl.Categorical,
    'device_type': pl.Int8,
    'device_conn_type': pl.Int8,
    'C14': pl.Int16,
    'C15': pl.Int16,
    'C16': pl.Int16,
    'C17': pl.Int16,
    'C18': pl.Int16,
    'C19': pl.Int16,
    'C20': pl.Int32,
    'C21': pl.Int16
}

# Load train and test datasets
train_df = pl.read_csv('/kaggle/input/avazu-ctr-prediction/train.gz', schema=train_schema)
test_df = pl.read_csv('/kaggle/input/avazu-ctr-prediction/test.gz', schema=test_schema)

# Step 1: Sample a much smaller fraction (0.1%) of the data to reduce size
train_df_sampled = train_df.sample(n=int(len(train_df) * 0.0001))  # Sample 0.01%
test_df_sampled = test_df.sample(n=int(len(test_df) * 0.0001))      # Sample 0.01%

# Step 2: Feature Engineering - Extract 'hour_of_day' and 'day_of_month' from 'hour'
def extract_hour_features(df):
    df = df.with_columns([
        (pl.col('hour').str.slice(6, 2).cast(pl.Int8)).alias('hour_of_day'),   # Extract hour
        (pl.col('hour').str.slice(4, 2).cast(pl.Int8)).alias('day_of_month')   # Extract day
    ])
    return df.drop('hour')

train_df_sampled = extract_hour_features(train_df_sampled)
test_df_sampled = extract_hour_features(test_df_sampled)

# Step 3: Remove unnecessary columns - Keep only a subset of important columns
important_columns = [
    'click', 'C1', 'banner_pos', 'site_id', 'site_domain', 'site_category',
    'app_id', 'app_domain', 'app_category', 'device_id', 'device_model', 
    'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'hour_of_day', 'day_of_month'
]

# Select only these columns
train_df_trimmed = train_df_sampled.select(important_columns)
test_df_trimmed = test_df_sampled.select([col for col in important_columns if col != 'click'])  # Exclude 'click' from test set

# Step 4: Encode categorical features using LabelEncoder
categorical_columns = ['site_id', 'site_domain', 'site_category', 'app_id', 'app_domain',
                       'app_category', 'device_id', 'device_model']

def label_encode_columns(df, columns):
    for col in columns:
        le = LabelEncoder()
        df = df.with_columns([(pl.Series(col, le.fit_transform(df[col].to_numpy()))).alias(col)])
    return df

# Apply label encoding to train and test sets
train_df_encoded = label_encode_columns(train_df_trimmed, categorical_columns)
test_df_encoded = label_encode_columns(test_df_trimmed, categorical_columns)

# Step 5: Separate features and labels for training
X_train = train_df_encoded.drop(['click']).to_pandas().values
y_train = train_df_encoded['click'].to_pandas().values

# Prepare Test Data
X_test = test_df_encoded.to_pandas().values

In [23]:
# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

Graph construction

In [24]:
import numpy as np
import torch

def create_sparse_graph(X, threshold=0.05):
    num_nodes = X.shape[0]
    
    # Create an array of possible edges (i, j) where i < j
    row_indices = np.repeat(np.arange(num_nodes), num_nodes - 1)
    col_indices = np.concatenate([np.delete(np.arange(num_nodes), i) for i in range(num_nodes)])
    
    # Generate random values to determine if an edge is created
    edge_probs = np.random.rand(len(row_indices))
    
    # Select edges where the random value is less than the threshold
    valid_edges = edge_probs < threshold
    
    # Get the selected edges
    edge_index = np.column_stack((row_indices[valid_edges], col_indices[valid_edges]))
    
    # Ensure valid edges
    if edge_index.size == 0:  # If no edges were added
        edge_index = np.array([[0, 1], [1, 0]])  # Default edge to avoid empty graph
    
    edge_index = torch.tensor(edge_index, dtype=torch.long).t()
    
    # Convert feature matrix to tensor
    X_tensor = torch.tensor(X, dtype=torch.float)
    
    return X_tensor, edge_index

In [25]:
X_train_tensor, edge_index = create_sparse_graph(X_train, threshold=0.1)  # Increase the threshold to reduce edge density
X_val_tensor, _ = create_sparse_graph(X_val, threshold=0.01)

GCN Model

In [26]:
class GCNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [27]:
def train_gcn_model(X_train, y_train, X_val, y_val, edge_index, input_dim, hidden_dim, output_dim, epochs=100, lr=0.01, batch_size=32):
    model = GCNModel(input_dim, hidden_dim, output_dim).to(device)
    criterion = nn.BCEWithLogitsLoss()  # Adjust based on your task (e.g., binary classification)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Convert labels to tensor and move to the correct device
    y_train_tensor = torch.tensor(y_train, dtype=torch.float).to(device)
    X_train = X_train.to(device)
    edge_index = edge_index.to(device)

    # Training loop
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass
        out = model(X_train, edge_index)
        
        # Compute loss
        loss = criterion(out.view(-1), y_train_tensor)
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

    return model

In [28]:
print(f"X_train_tensor shape: {X_train_tensor.shape}, dtype: {X_train_tensor.dtype}")
print(f"X_val_tensor shape: {X_val_tensor.shape}, dtype: {X_val_tensor.dtype}")
print(f"edge_index shape: {edge_index.shape}, dtype: {edge_index.dtype}")
print(f"y_train shape: {y_train.shape}, dtype: {y_train.dtype}")
print(f"Unique labels in y_train: {np.unique(y_train)}")  # Check if labels are 0 or 1

X_train_tensor shape: torch.Size([3233, 17]), dtype: torch.float32
X_val_tensor shape: torch.Size([809, 17]), dtype: torch.float32
edge_index shape: torch.Size([2, 1046157]), dtype: torch.int64
y_train shape: (3233,), dtype: int8
Unique labels in y_train: [0 1]


In [29]:
# Check for NaN or invalid values in y_train
print(f"y_train contains NaN: {np.isnan(y_train).any()}")
print(f"y_train contains values outside [0, 1]: {(y_train < 0).any() or (y_train > 1).any()}")

y_train contains NaN: False
y_train contains values outside [0, 1]: False


In [30]:
# Try converting to tensor without sending to device first
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

# Print tensor stats before sending to GPU
print(f"y_train_tensor stats - Min: {y_train_tensor.min()}, Max: {y_train_tensor.max()}")

# Now send to device
y_train_tensor = y_train_tensor.to(device)

y_train_tensor stats - Min: 0.0, Max: 1.0


In [31]:
# Convert y_train to torch.float32 for BCEWithLogitsLoss
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)

# Move tensors to device
X_train_tensor = X_train_tensor.to(device)
X_val_tensor = X_val_tensor.to(device)
edge_index = edge_index.to(device)

# Check after moving to device
# Assuming the tensors are already defined and moved to CPU
print(f"X_train_tensor is on device: {X_train_tensor.device}")
print(f"X_val_tensor is on device: {X_val_tensor.device}")
print(f"edge_index is on device: {edge_index.device}")
print(f"y_train_tensor is on device: {y_train_tensor.device}")

# Define input/output dimensions
input_dim = X_train_tensor.shape[1]
hidden_dim = 64  # You can adjust this
output_dim = 1   # Adjust based on your task

# Train the model
model = train_gcn_model(X_train_tensor, y_train_tensor, X_val_tensor, y_val, edge_index, input_dim, hidden_dim, output_dim)

X_train_tensor is on device: cpu
X_val_tensor is on device: cpu
edge_index is on device: cpu
y_train_tensor is on device: cpu


/tmp/ipykernel_30/1743774894.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train, dtype=torch.float).to(device)


Epoch [10/100], Loss: 509.5795
Epoch [20/100], Loss: 319.3154
Epoch [30/100], Loss: 22.3242
Epoch [40/100], Loss: 34.4552
Epoch [50/100], Loss: 182.1496
Epoch [60/100], Loss: 94.7406
Epoch [70/100], Loss: 63.6325
Epoch [80/100], Loss: 42.6895
Epoch [90/100], Loss: 198.0491
Epoch [100/100], Loss: 28.4504


In [32]:
print(f"X_val_tensor on device: {X_val_tensor.device}")
print(f"edge_index on device: {edge_index.device}")
print(f"y_val on device: {torch.tensor(y_val, dtype=torch.float32).device}")

X_val_tensor on device: cpu
edge_index on device: cpu
y_val on device: cpu


GCN Implementation

In [33]:
import torch
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Define device as CPU
device = torch.device("cpu")

# Reconstruct tensors on CPU to ensure a fresh start
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
edge_index = torch.tensor(edge_index, dtype=torch.int64)

# Ensure label tensors are in the correct format for BCEWithLogitsLoss
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# Check unique indices in edge_index
print(f"Unique indices in edge_index: {torch.unique(edge_index)}")
print(f"Maximum index in edge_index: {edge_index.max()}")
print(f"Number of nodes in X_val_tensor: {X_val_tensor.shape[0]}")

# Filter out invalid edges
max_index = X_val_tensor.shape[0] - 1
valid_edges = edge_index[:, (edge_index[0] <= max_index) & (edge_index[1] <= max_index)]

# Print filtered edge index information
print(f"Filtered edge_index: shape = {valid_edges.shape}, unique indices = {torch.unique(valid_edges)}")

# Evaluation on CPU
model.eval()
with torch.no_grad():
    val_out = model(X_val_tensor, valid_edges)  # Use filtered edge index
    val_out = torch.sigmoid(val_out)

    # Convert predictions and labels to numpy arrays for metric calculation
    val_out_np = val_out.numpy()
    y_val_np = y_val_tensor.numpy()

    # Apply binary threshold for classification
    y_pred = (val_out_np >= 0.5).astype(int)
    accuracy = (y_pred.flatten() == y_val_np).mean()
    print(f"Validation Accuracy on CPU: {accuracy:.4f}")

    # Additional metrics using sklearn
    precision = precision_score(y_val_np, y_pred)
    recall = recall_score(y_val_np, y_pred)
    f1 = f1_score(y_val_np, y_pred)


Unique indices in edge_index: tensor([   0,    1,    2,  ..., 3230, 3231, 3232])
Maximum index in edge_index: 3232
Number of nodes in X_val_tensor: 809
Filtered edge_index: shape = torch.Size([2, 65334]), unique indices = tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139

/tmp/ipykernel_30/861299152.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(edge_index, dtype=torch.int64)
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Improving the Model:

In [34]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_undirected
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, TensorDataset
from torch_geometric.utils import add_self_loops

# Define the enhanced GCN model with multi-layer and specified embedding
class EnhancedGCNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=3, dropout=0.5):
        super(EnhancedGCNModel, self).__init__()
        self.embedding_layer = nn.Linear(input_dim, 8)  # Embedding layer
        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList()
        
        # Define multiple GCN layers as per the paper
        for i in range(num_layers):
            in_dim = 8 if i == 0 else hidden_dim
            self.convs.append(GCNConv(in_dim, hidden_dim))
            self.bns.append(nn.BatchNorm1d(hidden_dim))
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.ReLU()

    def forward(self, x, edge_index):
        x = self.embedding_layer(x)
        
        for conv, bn in zip(self.convs, self.bns):
            x = conv(x, edge_index)
            x = bn(x)
            x = self.activation(x)
            x = self.dropout(x)
        
        x = self.fc(x)
        return x

# Training function with batch-wise edge filtering
def train_model(model, train_loader, edge_index, optimizer, criterion, num_epochs=100):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch_x, batch_y in train_loader:
            batch_nodes = batch_x.shape[0]
            
            # Filter edge_index for the batch to only contain within-batch edges
            mask = (edge_index[0] < batch_nodes) & (edge_index[1] < batch_nodes)
            batch_edge_index = edge_index[:, mask]
            
            optimizer.zero_grad()
            out = model(batch_x, batch_edge_index)
            loss = criterion(out.view(-1), batch_y.float())
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # Print epoch loss every 10 epochs
        if epoch % 10 == 0:
            avg_loss = total_loss / len(train_loader)
            print(f'Epoch [{epoch}/{num_epochs}], Loss: {avg_loss:.4f}')
            
def evaluate_model(model, X_val_tensor, edge_index, y_val_tensor):
    model.eval()
    with torch.no_grad():
        # Map nodes in X_val_tensor to the corresponding range in edge_index
        unique_nodes = torch.arange(X_val_tensor.size(0), device=edge_index.device)
        node_map = {n.item(): i for i, n in enumerate(unique_nodes)}

        # Filter edge_index to only include edges within the validation set
        mask = (edge_index[0] < X_val_tensor.size(0)) & (edge_index[1] < X_val_tensor.size(0))
        filtered_edge_index = edge_index[:, mask]
        
        # Map the edges to the new subset of nodes
        filtered_edge_index = torch.stack([
            torch.tensor([node_map.get(n.item(), -1) for n in filtered_edge_index[0]]),
            torch.tensor([node_map.get(n.item(), -1) for n in filtered_edge_index[1]])
        ], dim=0)
        
        # Remove any edges with -1 (those outside the mapped range)
        valid_edges = (filtered_edge_index != -1).all(dim=0)
        filtered_edge_index = filtered_edge_index[:, valid_edges]
        
        # Forward pass with filtered edge_index
        out = model(X_val_tensor, filtered_edge_index)
        out_sigmoid = torch.sigmoid(out).view(-1).cpu().numpy()
        y_true = y_val_tensor.cpu().numpy()

        # Calculate evaluation metrics
        auc = roc_auc_score(y_true, out_sigmoid)
        logloss = log_loss(y_true, out_sigmoid)
        accuracy = accuracy_score(y_true, (out_sigmoid >= 0.5).astype(int))
       

        print(f'Validation AUC: {auc:.4f}')
        print(f'Validation Log Loss: {logloss:.4f}')
        print(f'Validation Accuracy: {accuracy:.4f}')
        
        return auc, logloss, accuracy
    # Prepare data for training and validation
# Assume X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, edge_index are defined elsewhere
# Update edge_index to undirected and with self-loops
edge_index = to_undirected(edge_index)
edge_index, _ = add_self_loops(edge_index)

# Define parameters
input_dim = X_train_tensor.shape[1]
hidden_dim = 64
output_dim = 1
batch_size = 1024
learning_rate = 0.01
num_epochs = 100

# Instantiate model, optimizer, and loss function
model = EnhancedGCNModel(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
criterion = nn.BCEWithLogitsLoss()

# Set up DataLoader for batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Train the model
train_model(model, train_loader, edge_index, optimizer, criterion, num_epochs)

# Evaluate the model on validation data
evaluate_model(model, X_val_tensor, edge_index, y_val_tensor)

Epoch [0/100], Loss: 0.5632
Epoch [10/100], Loss: 0.4671
Epoch [20/100], Loss: 0.5016
Epoch [30/100], Loss: 0.4808
Epoch [40/100], Loss: 0.4800
Epoch [50/100], Loss: 0.4616
Epoch [60/100], Loss: 0.4619
Epoch [70/100], Loss: 0.4655
Epoch [80/100], Loss: 0.4669
Epoch [90/100], Loss: 0.4685
Validation AUC: 0.5193
Validation Log Loss: 1.3986
Validation Accuracy: 0.8405


(0.5192772457820338, 1.3985619840181394, 0.8405438813349815)

GNN model implementation

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import SAGEConv  # Change to other layers if needed
from torch_geometric.utils import to_undirected, add_self_loops
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score
from torch.utils.data import DataLoader, TensorDataset

# Define the generic GNN model with multi-layer capability
class EnhancedGNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=3, dropout=0.5):
        super(EnhancedGNNModel, self).__init__()
        self.embedding_layer = nn.Linear(input_dim, 8)  # Embedding layer
        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList()
        
        # Define multiple GNN layers
        for i in range(num_layers):
            in_dim = 8 if i == 0 else hidden_dim
            self.convs.append(SAGEConv(in_dim, hidden_dim))
            self.bns.append(nn.BatchNorm1d(hidden_dim))
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.ReLU()

    def forward(self, x, edge_index):
        x = self.embedding_layer(x)
        
        for conv, bn in zip(self.convs, self.bns):
            x = conv(x, edge_index)
            x = bn(x)
            x = self.activation(x)
            x = self.dropout(x)
        
        x = self.fc(x)
        return x

# Training function with batch-wise edge filtering
def train_model(model, train_loader, edge_index, optimizer, criterion, num_epochs=100):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch_x, batch_y in train_loader:
            batch_nodes = batch_x.shape[0]
            
            # Filter edge_index for the batch to only contain within-batch edges
            mask = (edge_index[0] < batch_nodes) & (edge_index[1] < batch_nodes)
            batch_edge_index = edge_index[:, mask]
            
            optimizer.zero_grad()
            out = model(batch_x, batch_edge_index)
            loss = criterion(out.view(-1), batch_y.float())
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # Print epoch loss every 10 epochs
        if epoch % 10 == 0:
            avg_loss = total_loss / len(train_loader)
            print(f'Epoch [{epoch}/{num_epochs}], Loss: {avg_loss:.4f}')

def evaluate_model(model, X_val_tensor, edge_index, y_val_tensor):
    model.eval()
    with torch.no_grad():
        # Map node indices in X_val_tensor to global indices
        node_map = {node.item(): i for i, node in enumerate(torch.arange(X_val_tensor.shape[0]))}
        
        # Filter edge_index for the batch to include only nodes in X_val_tensor
        mask = (edge_index[0] < X_val_tensor.shape[0]) & (edge_index[1] < X_val_tensor.shape[0])
        filtered_edge_index = edge_index[:, mask]
        
        out = model(X_val_tensor, filtered_edge_index)
        out_sigmoid = torch.sigmoid(out).view(-1).cpu().numpy()
        y_true = y_val_tensor.cpu().numpy()

        # Calculate only AUC, Log Loss, and Accuracy
        auc = roc_auc_score(y_true, out_sigmoid)
        logloss = log_loss(y_true, out_sigmoid)
        accuracy = accuracy_score(y_true, (out_sigmoid >= 0.5).astype(int))
        
        print(f'Validation AUC: {auc:.4f}')
        print(f'Validation Log Loss: {logloss:.4f}')
        print(f'Validation Accuracy: {accuracy:.4f}')
        
        return auc, logloss, accuracy


# Prepare data for training and validation
# Assume X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, edge_index are defined elsewhere
# Update edge_index to undirected and with self-loops
edge_index = to_undirected(edge_index)
edge_index, _ = add_self_loops(edge_index)

# Define parameters
input_dim = X_train_tensor.shape[1]
hidden_dim = 64
output_dim = 1
batch_size = 1024
learning_rate = 0.01
num_epochs = 100

# Instantiate model, optimizer, and loss function
model = EnhancedGNNModel(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
criterion = nn.BCEWithLogitsLoss()

# Set up DataLoader for batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Train the model
train_model(model, train_loader, edge_index, optimizer, criterion, num_epochs)

# Evaluate the model on validation data
evaluate_model(model, X_val_tensor, edge_index, y_val_tensor)

Epoch [0/100], Loss: 0.5794
Epoch [10/100], Loss: 0.4773
Epoch [20/100], Loss: 0.4758
Epoch [30/100], Loss: 0.4745
Epoch [40/100], Loss: 0.4739
Epoch [50/100], Loss: 0.4559
Epoch [60/100], Loss: 0.4585
Epoch [70/100], Loss: 0.4354
Epoch [80/100], Loss: 0.4436
Epoch [90/100], Loss: 0.4489
Validation AUC: 0.5309
Validation Log Loss: 0.4545
Validation Accuracy: 0.8405


(0.530859553123575, 0.4544685356991035, 0.8405438813349815)